In [5]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os
import re

In [28]:
import numpy as np
import torch
import os
import torch.nn.functional as F  # Import F for padding

# Path to the directory containing the saved NumPy arrays
NUMPY_DIR = "keystroke_spectrograms/numpy_arrays"

# Function to load all spectrograms from the directory
def load_spectrograms_from_directory(directory):
    spectrograms = []
    keys = []
    widths = []
    filenames = os.listdir(directory)

    # Loop through each file in the directory
    for filename in filenames:
        if filename.endswith('.npy'):
            file_path = os.path.join(directory, filename)

            # Load the NumPy array from file
            spectrogram = np.load(file_path)

            widths.append(spectrogram.shape[1])

            key = re.search(r"keystroke_\d+_([A-Za-z])\.npy", filename).group(1)

            # Convert the spectrogram to a PyTorch tensor
            # Adding a channel dimension (1, height, width) for CNN input
            spectrogram_tensor = torch.tensor(spectrogram).unsqueeze(0).float()  # Shape: (1, n_freq_bins, n_time_bins)

            # next, add the tensor to a list
            spectrograms.append(spectrogram_tensor)
            keys.append(key)

    return spectrograms, keys, max(widths)

# Load all spectrograms from the specified directory
spectrogram_tensors, keys, max_width = load_spectrograms_from_directory(NUMPY_DIR)

assert len(spectrogram_tensors) == len(keys), "The number of spectrograms and keys do not match!"


## Need to pad the spectrograms to the same width

# Padding function
def pad_tensor(tensor, max_width):
    # Calculate padding size (for the width dimension)
    current_width = tensor.shape[2]  # Index 2 is for the time dimension (width)
    if current_width < max_width:
        padding = max_width - current_width
        # Pad with zeros on the right (width dimension)
        return F.pad(tensor, (0, padding))  # Pad (left, right)
    return tensor

# Apply padding to all spectrograms
for i in range(len(spectrogram_tensors)):
    spectrogram_tensors[i] = pad_tensor(spectrogram_tensors[i], max_width)

# Verify padding
for i, spectrogram in enumerate(spectrogram_tensors):
    print(f"Shape of spectrogram {i}: {spectrogram.shape}")


Shape of spectrogram 0: torch.Size([1, 129, 101])
Shape of spectrogram 1: torch.Size([1, 129, 101])
Shape of spectrogram 2: torch.Size([1, 129, 101])
Shape of spectrogram 3: torch.Size([1, 129, 101])
Shape of spectrogram 4: torch.Size([1, 129, 101])
Shape of spectrogram 5: torch.Size([1, 129, 101])
Shape of spectrogram 6: torch.Size([1, 129, 101])
Shape of spectrogram 7: torch.Size([1, 129, 101])
Shape of spectrogram 8: torch.Size([1, 129, 101])
Shape of spectrogram 9: torch.Size([1, 129, 101])
Shape of spectrogram 10: torch.Size([1, 129, 101])
Shape of spectrogram 11: torch.Size([1, 129, 101])
Shape of spectrogram 12: torch.Size([1, 129, 101])
Shape of spectrogram 13: torch.Size([1, 129, 101])
Shape of spectrogram 14: torch.Size([1, 129, 101])
Shape of spectrogram 15: torch.Size([1, 129, 101])
Shape of spectrogram 16: torch.Size([1, 129, 101])
Shape of spectrogram 17: torch.Size([1, 129, 101])
Shape of spectrogram 18: torch.Size([1, 129, 101])
Shape of spectrogram 19: torch.Size([1, 1

In [29]:
class KeystrokeDataset(Dataset):
    def __init__(self, spectrograms_tensors, labels):
        self.spectrograms = spectrograms_tensors
        self.labels = labels

    def __len__(self):
        return len(self.spectrograms)

    def __getitem__(self, idx):
        return self.spectrograms[idx], self.labels[idx]
    
tensors,labels, _ = load_spectrograms_from_directory(NUMPY_DIR)
tensors = spectrogram_tensors # adjusting the tensors to the padded spectrograms
train_dataset = KeystrokeDataset(spectrogram_tensors,labels)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


#test_dataset = KeystrokeDataset(test_spectrograms_tensors, test_labels)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [30]:
class KeystrokeCNN(nn.Module):
    def __init__(self):
        super(KeystrokeCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # 1 input channel (grayscale)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 7 * 7, 512)  # Adjust size based on input dimensions
        self.fc2 = nn.Linear(512, 26)  # 26 output classes (A-Z)
        
    def forward(self, x):
        # Forward pass through convolutional layers
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        
        # Flatten the output from convolutional layers to feed into fully connected layers
        x = x.view(-1, 128 * 7 * 7)  # Adjust based on input size (H, W)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

# Initialize the model
model = KeystrokeCNN()




In [31]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [32]:
# Function to train the model
def train(model, train_loader, criterion, optimizer, epochs=5):
    model.train()  # Set the model to training mode
    
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 200 == 199:  # Print every 200 batches
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 200:.3f}")
                running_loss = 0.0

# Train the model
train(model, train_loader, criterion, optimizer, epochs=5)


RuntimeError: shape '[-1, 6272]' is invalid for input of size 688128